In [ ]:
!pip install autogen-agentchat~=0.2
!pip install numpy==1.26.4 flaml==2.1.2

In [ ]:
import os

from google.colab import userdata
from typing import Annotated


In [ ]:
api_key = os.environ.get("API_KEY")


In [ ]:
!pip install autogen
import autogen

In [ ]:
system_summery = """INPUT YOUR SYSTEM SUMMERY HERE"""

In [ ]:
task = """Generate five or more ethics requirements, focusing on Transparency, Fairness, and Data, in the form of user stories with acceptance criteria, for building a system with the following summary.: """ + system_summery
config_list = {
        "cache_seed": None,
        "model": 'gemini-2.0-flash',
        "base_url": "https://generativelanguage.googleapis.com/v1beta",
        "temperature": 1,
    }
advocate = autogen.AssistantAgent(
    name="Advocate",
    llm_config=config_list,
    max_consecutive_auto_reply=4,
    system_message="""
      You are an ethics advocate, known for your commitment to spotting ethical issues related to each stakeholder involved. Values you focus on are:
      (1) Transparency, including: Internal and External Transparency, Explainability, Communication, Documentation, Traceability, System reliability
      (2) Fairness, including: Accessibility, Inclusion,
      (3) Data, including: Privacy, Data quality, Access to data
      Your task is to scrutinize the existing requirements for any potential violation of these values or suggest new ethics-related requirements.
      Think about each of the ethical challenges and its subpoints carefully and solve them one step at a time.
      Acceptance criteria must be specific regarding ethical issues. No use of general words like (all ages, all skin tones, all genders) is allowed. Enumerations of such categories are required.
      Do not introduce frameworks; just a requirements draft. If you don't have any additional suggestions or comments, state that: "The requirements are approved from an ethics point of view".
    """,
)
qualityA = autogen.AssistantAgent(
    name="Quality_Assurance",
    llm_config=config_list,
    system_message="""
    You are a quality assurance professional, known for your commitment to spotting quality issues.
    Quality traits that we want are for the requirement to be atomic, minimal, estimatable, Problem-oriented, and Unambiguous.
    Your task is to scrutinize requirements for any potential violation of these quality issues.
    Think about each of the quality criteria carefully and report violations one point at a time, if present.
    If you don't have any major issues or comments state that: "The requirements are approved from a quality point of view," then repeat the requirements and state they need ethical review.
    """,
)

engineer = autogen.AssistantAgent(
    name="engineer",
    llm_config=config_list,
    system_message="""
    You are a requirements engineer, known for your precision and thoughtfulness.
    You should improve the quality of the content based on the feedback you get from critics and ethics advocates. Always carry on with the latest set of agreed requirements.
    """,
)

document = autogen.AssistantAgent(
    name="documenter",
    llm_config=config_list,
    system_message="""
    You are an assistant responsible for printing out or listing the last version of the agreed-upon requirement.
    Do not just comment on them. List the final requirements as a bulleted list, along with their acceptance criteria.
    """,
)


def custom_speaker_selection_func(last_speaker: autogen.Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages
    print("Messages: ", messages[-1])
    if last_speaker is qualityA:
        if "The requirements are approved from a quality point of view" in messages[-1]["content"]:
            return advocate
        else:
            return engineer
    elif last_speaker is engineer:
      if len(messages) < 2 or messages[-2]["name"] == qualityA.name:
         if "The requirements are approved from a quality point of view" in messages[-1]["content"]:
            return advocate
         return qualityA
      elif "The requirements are approved from an ethics point of view" in messages[-2]["content"]:
          return document
      elif count_name_occurrences(messages,advocate.name) == 2:
          return document
      else:
          return advocate

    elif last_speaker is advocate:
      if "The requirements are approved from an ethics point of view" in messages[-1]["content"]:
          # If the last message is approved, let the engineer to speak
          return document
      else:
          return engineer
    elif last_speaker is document:
      return user_proxy
    return document

def count_name_occurrences(messages, target_name):
    count = sum(1 for msg in messages if msg.get("name") == target_name)
    print(f"Occurrences of '{target_name}': {count}")
    return count


user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",
    llm_config=config_list,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "my_code",
        "use_docker": False,
    }
)


groupchat = autogen.GroupChat( agents=[user_proxy, engineer, advocate, qualityA,document],
    messages=[],
    max_round=20,
    speaker_selection_method=custom_speaker_selection_func,

)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=config_list)

results = manager.initiate_chat(
    engineer,
    message=task,
    summary_method="last_msg",
    max_turns=20,

)